In [ ]:
import pandas as pd
from pandas import DataFrame
import portfolio
import stats
from visualisation import plot_portfolio_value

In [ ]:
def calculate_sma(df: DataFrame, window):
    return df['Close'].rolling(window=window).mean()

In [ ]:
# Read in the pre-processed data
ticker = "AAPL"
data_path = f"../data/{ticker}_processed_hourly_data.csv"
data = pd.read_csv(data_path, index_col='Datetime', parse_dates=True)

In [ ]:
# Parameters
short_window = 10
long_window = 100

# Time period (for days set it to 252 and for hours to 1638)
time_period = 1638

# Calculate the short and long-term SMA's
data['SMA_short'] = calculate_sma(data, short_window)
data['SMA_long'] = calculate_sma(data, long_window)

# Filter out the rows that will be null and add column to track value
data = data[data['SMA_long'].notna()]
data['Portfolio Value'] = None
data['Actions'] = None

# Run the SMA strategy
portfolio = portfolio.Portfolio(initial_cash=100000)

stop_loss_percentage = 0.02
take_profit_percentage = 0.05

In [ ]:
for index, row in data.iterrows():
    current_price = row['Close']
    data.at[index, 'Portfolio Value'] = portfolio.get_value(current_price)

    # Stop loss and take-profit logic
    if portfolio.holdings > 0 and portfolio.last_buy_price:
        if current_price < (portfolio.last_buy_price * (1 - stop_loss_percentage)):
            portfolio.sell(current_price, portfolio.holdings)
            row['Actions'] = -1
            
        elif current_price > (portfolio.last_buy_price * (1 + take_profit_percentage)):
            portfolio.sell(current_price, portfolio.holdings)
            row['Actions'] = -1
            

    # SMA logic here
    if row['SMA_short'] > row['SMA_long'] and not portfolio.holdings:
        portfolio.buy(row['Close'], 250)
        row['Actions'] = 1
        
    elif row['SMA_short'] < row['SMA_long'] and portfolio.holdings:
        portfolio.sell(row['Close'], 250)
        row['Actions'] = -1
        

# Calculate the percentage change (for analysis)
data['Returns'] = data['Portfolio Value'].pct_change().dropna()

# Get the final value at the last day of the data
final_value = portfolio.get_value(data.iloc[-1]['Close'])
print(f"Final Portfolio Value = {final_value}")

In [ ]:
years = ((data.index[-1] - data.index[0]).days / 365)

cagr = stats.calculate_compound_annual_growth_rate(data.iloc[0]['Portfolio Value'], data.iloc[-1]['Portfolio Value'], years)
sharpe_ratio = stats.calculate_sharpe_ratio(data['Returns'], periods_per_year=time_period)
max_drawdown = stats.calculate_max_drawdown(data['Portfolio Value'].dropna().tolist())
volatility = stats.calculate_volatility(data['Returns'], periods_per_year=time_period)

In [ ]:
print(f"Compound Annual Growth Rate: {cagr}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"max_drawdown: {max_drawdown}")
print(f"volatility: {volatility}")

In [ ]:
plot_portfolio_value(data, "Simple Moving Average")